# Covid19 Greece Analysis

## Install the dependencies

In [7]:
!pip install pandas bokeh numpy scipy folium

## Get an overview of the situation

In [8]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.io import show, output_notebook
from bokeh.layouts import row

output_notebook()

names = ["city", "latitude", "longitude", "infections"]
df = pd.read_csv(
    'cities.csv',
    names=names,
)

data = df.loc[:, ['city', 'infections']]
data = data.sort_values(ascending=True, by=['infections'])

p = figure(x_range=data['city'], title="Μολύνσεις ανά πόλη",
           x_axis_label="City", y_axis_label="Αριθμός Μολυσμένων", width=900)

p.vbar(x=data['city'], top=data['infections'], width=0.9)
p.tools = [] # disable zoom / pan
p.add_tools(HoverTool(tooltips=[("City", "@x"), ("Infected", "@top")]))
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.toolbar.logo = None
p.toolbar_location = None


show(p)

Loading BokehJS ...

## Visualize in a Map

In [9]:
import folium

MAGNIFICATION = 100

# map centered at Athens
map = folium.Map(
    location=[37.9838, 23.7275],
    width=750,
    height=750
)

for i, city in df.dropna().iterrows():
    # add a tooltip showing the number of infections on top
    popup = folium.Popup(str(city["infections"]), sticky=True, show=True)
    folium.Circle(
        location=[
            city["latitude"],
            city["longitude"]
        ],
        radius=city["infections"] * MAGNIFICATION,
        color="red",
        sticky=True,
        show=True,
        fill=True,  # Set fill to True
        fill_opacity=0.7
    ).add_child(popup).add_to(map)

sw = df[['latitude', 'longitude']].min().values.tolist()
ne = df[['latitude', 'longitude']].max().values.tolist()
map.fit_bounds([sw, ne])

map

# Predictions

We fit the data to a Sigmoid curve

In [10]:
names = ["city", "latitude", "longitude", "infections"]
loc = pd.read_csv(
    'cities.csv',
    names=names,
)

# load the data
names = ["date", "infections"]
ts = pd.read_csv(
    'greece.csv',
    names=names,
)
ts['date'] = pd.to_datetime(ts['date'], dayfirst=True)

# cumulative infections
ts['cumulative-infections'] = ts['infections'].cumsum()

# infections
infections_graph = figure(title = "Μολύνσεις ανά ημέρα", x_axis_label = "Date", y_axis_label = "Number infected", x_axis_type="datetime")

# Add a quad glyph
infections_graph.line(ts["date"], ts["infections"])


# cumulative infections
total_infected = figure(title = "Συνολικές Μολύνσεις", x_axis_label = "Date", y_axis_label = "Number infected", x_axis_type="datetime")

# Add a quad glyph
total_infected.line(ts["date"], ts["cumulative-infections"])

infections_graph.add_tools(HoverTool(tooltips=[("Date", "@x"), ("New infections", "@y")]))
total_infected.add_tools(HoverTool(tooltips=[("Date", "@x"), ("Total Infected", "@y")]))

show(row(infections_graph, total_infected))

# The Logistic Curve

Explain population growth

In [11]:
import numpy as np

# f(x) = \frac{L}{1 + e^{-k(x-x0)}}
def sigmoid(x, L, x0, k):
    y = L / (1 + np.exp(-k*(x-x0)))
    return y

def sigmoid_with_growth(growth):
    return lambda x, L, x0: sigmoid(x, L, x0, growth)

x = np.linspace(-10, 10, 1000)
y = sigmoid(x, 1, 0.5, 1)

# cumulative infections
logistic_curve = figure(title = "Logistic Curve", x_axis_label = "time", y_axis_label = "Population Growth", x_axis_type="datetime")
logistic_curve.xaxis.fixed_location = 0
logistic_curve.add_tools(HoverTool(tooltips=[("Time", "@x"), ("Population Size", "@y")]))

# Add a quad glyph
logistic_curve.line(x, y)

show(logistic_curve)

# Predictions and Curve Fitting

Using Italy as reference, we will use growth rates 0.23, 0.235, 0.24

In [26]:
from scipy.optimize import curve_fit
from bokeh.io import push_notebook

first_infection = ts['date'].loc[0]

SAMPLES = 15

# get days since first infection
x_train = (ts['date'] - first_infection).dt.days[:SAMPLES]
y_train = list(ts['cumulative-infections'])[:SAMPLES]


    
sigmoids = figure(y_range=(0,7000), title = "Predictions", x_axis_label = "Date", y_axis_label = "Total Infections", x_axis_type="datetime", width =800)


# try to project 90 days since the first incident
real = sigmoids.asterisk(ts['date'][:SAMPLES], ts['cumulative-infections'][:SAMPLES])
pred = sigmoids.circle_x(ts['date'][SAMPLES:], ts['cumulative-infections'][SAMPLES:], line_color="#f0027f")
sigmoids.add_tools(HoverTool(tooltips=[("Date", "@x"), ("Total infections", "@y")], formatters={'DateTime': 'datetime'}))


x = range(0, 90) # 90 days forecast
x_days = first_infection + pd.to_timedelta(x, unit = 'D')
growth_rate = 0.23
popt, pcov = curve_fit(sigmoid_with_growth(growth_rate), x_train, y_train, maxfev=10000)
y = sigmoid(x, *popt, growth_rate)
r = sigmoids.line(x_days, y)

def render(k=0.23, SAMPLES = 15):
    x_train = (ts['date'] - first_infection).dt.days[:SAMPLES]
    y_train = list(ts['cumulative-infections'])[:SAMPLES]
    
    # update the trained values
    real.data_source.data = {'x': ts['date'][:SAMPLES], 'y':  ts['cumulative-infections'][:SAMPLES] }
    
    # update the new values
    pred.data_source.data = {'x': ts['date'][SAMPLES:], 'y': ts['cumulative-infections'][SAMPLES:] }
    
    popt, pcov = curve_fit(sigmoid_with_growth(k), x_train, y_train, maxfev=10000)
    r.data_source.data['y'] = sigmoid(x, *popt, k)
    push_notebook()
    

show(sigmoids, notebook_handle=True)
from ipywidgets import interact
interact(render, k=(0.22,0.35,0.0015), samples = (10, len(ts)))

interactive(children=(FloatSlider(value=0.23, description='k', max=0.35, min=0.22, step=0.0015), IntSlider(val…

<function __main__.render(k=0.23, SAMPLES=15)>